# Analysis

Notebook reads the list of the selected authors from `res/selected/author_list` and loads their text data from book and genereted resources available in `res` directory.
Displayed data:
- Notebook displays the number of words per each source-author pair.
- Analysis on chunk of `configuration.analysis_size` size
    - Average word length
    - Average sentence length
    - Unique words count
    - Top 10 function words

Further work on that notebook is planned.

*Note: the list of function words are taken from [functionwords](https://pypi.org/project/functionwords/) package.*

In [1]:
from src import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Projects\Thesis\.venv\Lib\site-packages\pydantic\_internal\_fields.py:151: UserWarning: Field "model_dirs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
configuration = Configuration()

In [3]:
authors = []
for author_name in FileUtils.read_authors(configuration.selected_authors_filepath):
    author = Author(name=author_name)
    author.read_book_collection(configuration.all_book_data_directory)
    author.read_generated_texts(configuration.model_dirs)
    authors.append(author)

In [4]:
analysis = Analysis(configuration.analysis_size)
analysis_data = analysis.analyze(authors)

In [5]:
AnalysisVisualization().visualize(analysis_data)

c:\Projects\Thesis\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:322: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.


c:\Projects\Thesis\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:572: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




In [6]:
for collection_name, data in analysis_data.items():
    if collection_name == 'books':
        continue
    print("")
    for author_data in data:
        print(f"& {round(author_data.word_count / 1000)}k", end=" ")


& 267k & 291k & 310k & 303k & 273k & 290k & 299k & 294k & 290k & 300k 
& 322k & 340k & 342k & 302k & 253k & 276k & 269k & 260k & 251k & 262k 
& 296k & 311k & 318k & 280k & 252k & 278k & 286k & 279k & 278k & 263k 
& 251k & 251k & 289k & 260k & 252k & 266k & 252k & 251k & 251k & 251k 
& 689k & 630k & 608k & 559k & 558k & 547k & 589k & 481k & 470k & 608k 